In [ ]:
from flask import Flask
from flask import request
import nltk.stem.porter as stemmer
import json
import requests

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

@app.route("/search", methods=["POST"])
def search():
    j = request.json
    query = j['query']
    
    response_analyze = requests.post('http://127.0.0.1:13533/analyze', json={'data' : query})
    parsed_analyze = json.loads(response_analyze.text)
    
    response_reverceindex = requests.post('http://127.0.0.1:13538/reverceindex', json={'data': parsed_analyze['words']})
    parsed_reverceindex = json.loads(response_reverceindex.text)
    
    response_ranked_docs = requests.post('http://127.0.0.1:13541/rank', json={'data': parsed_reverceindex['precessed_data']})
    parsed_ranked_docs = json.loads(response_ranked_docs.text)
    
    return json.dumps({"status":"ok", "processed_data" : parsed_ranked_docs['ranked_docs']})
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13565)

 * Running on http://0.0.0.0:13565/ (Press CTRL+C to quit)
[2018-11-02 14:17:10,884] ERROR in app: Exception on /search [POST]
Traceback (most recent call last):
  File "C:\Users\busov\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\busov\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\busov\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\busov\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\busov\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\busov\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-6